## Introducción
Este cuaderno se centra en el procesamiento y análisis de un conjunto de datos de melanoma. Incluye pasos para la carga de datos, preprocesamiento, entrenamiento del modelo y evaluación.

### Indice
* [Model](##red-neuronal-simple)

In [ ]:
%matplotlib inline

# imports
import torch
import torch.nn as nn

import sklearn.metrics as metrics
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

from torchvision import transforms
from torch.utils.data import DataLoader
from PIL import Image
import os

from sklearn.model_selection import train_test_split

In [ ]:
import torch
# Configuración del dispositivo CUDA
print(torch.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Dispositivo configurado para usar: {device}")

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

In [ ]:
# consts
cats = ["MEL","NV","BCC","AK","BKL","DF","VASC","SCC","UNK"]

base_path_csv = "./dataset"
base_path_img = "./dataset"

random_state=1

train_percent = .7
val_percent = .2
test_percent = .1



## Configuración
Las siguientes celdas importan las librerías necesarias y definen constantes utilizadas a lo largo del cuaderno.

In [ ]:
csv = pd.read_csv(f"{base_path_csv}/ISIC_2019_Training_GroundTruth.csv")

print("> Cantidad de elementos", csv.count(axis=1).size )
print("> Head de los datos del csv")
print(csv.head())

In [ ]:
header = list(csv.columns)
header.remove("image")

print("> Categorias")
print(header)

In [ ]:

data = pd.DataFrame({"img":[], "cat": []}, dtype=int)
rows_list = []
for entry in csv.values:
    new_row = {"img": entry[0], "cat": np.where(entry==1.0)[0][0]-1}
    rows_list.append(new_row)

data = pd.DataFrame(rows_list)


print("> Cantidad de filas", csv.count(axis=1).size)
print("> Head de los datos en el formato que usa pytorch")
print(data.head())

## Carga de Datos y Exploración Inicial
El conjunto de datos se carga desde un archivo CSV, y se realiza una exploración inicial para entender su estructura y contenidos.

In [ ]:
counter = np.zeros(len(cats), dtype=int)

for elm in data.values:
    counter[int(elm[1])]+=1

print("> Cantidad de imagenes de cada tipo:")
print(counter)

In [ ]:
# counter = list(filter(lambda elm: elm > 0, counter))
# min_cat_size = min(counter)

min_cat_size = data.cat.value_counts().min()

print("> Cantidad mínima de imagenes de un tipo:")
print(min_cat_size)

In [ ]:
data = pd.DataFrame(data.groupby("cat").apply(lambda cat: cat.sample(min_cat_size, random_state=random_state)).reset_index(drop=True))

print("> Cantidad de elemntos:", data.count(axis=1).size )
print("> Datos equilibrados")
print(data.head())

In [ ]:
data = data.sample(frac=1, random_state=random_state)

print("> cantidad de elementos", data.count(axis=1).size)
print("> datos barajados")
print(data.head())

## Preprocesamiento y Análisis de Datos
Estas celdas manejan el preprocesamiento de datos, incluyendo la limpieza, equilibrio y preparación para el aprendizaje automático.

In [ ]:
counter = np.zeros(len(cats), dtype=int)

for elm in data.values:
    counter[int(elm[1])]+=1

print("> Cantidad de imagenes de cada tipo:")
print(counter)

In [ ]:
# data.to_csv(f"{base_path_csv}/balanced_data.csv", index=False)

In [ ]:
train_data, tmp = train_test_split(data, train_size=train_percent, stratify=data['cat'], shuffle=True, random_state=random_state)
val_data, test_data = train_test_split(tmp, test_size=test_percent/(test_percent+val_percent), stratify=tmp['cat'], shuffle=True, random_state=random_state)


print("> train", train_data.count(axis=1).size)
print(test_data.head())
print()

print("> val", val_data.count(axis=1).size)
print(val_data.head())
print()

print("> test", test_data.count(axis=1).size)
print(test_data.head())
print()

In [ ]:
counter = np.zeros(len(cats), dtype=int)
for elm in train_data.values:
    counter[int(elm[1])]+=1

print("> train", train_data.count(axis=1).size, train_data.count(axis=1).size/data.count(axis=1).size)
print(counter, f"({counter[0]/train_data.count(axis=1).size})")
print()

counter = np.zeros(len(cats), dtype=int)
for elm in val_data.values:
    counter[int(elm[1])]+=1

print("> val", val_data.count(axis=1).size, val_data.count(axis=1).size/data.count(axis=1).size)
print(counter, f"({counter[0]/val_data.count(axis=1).size})")
print()

counter = np.zeros(len(cats), dtype=int)
for elm in test_data.values:
    counter[int(elm[1])]+=1

print("> test", test_data.count(axis=1).size, test_data.count(axis=1).size/data.count(axis=1).size)
print(counter, f"({counter[0]/test_data.count(axis=1).size})")
print()

## División de Datos para el Entrenamiento del Modelo
El conjunto de datos se divide en conjuntos de entrenamiento, validación y prueba para prepararse para el entrenamiento del modelo.

In [ ]:
# train_data.to_csv(f"{base_path_csv}/train_data.csv", index=False)
# val_data.to_csv(f"{base_path_csv}/val_data.csv", index=False)
# test_data.to_csv(f"{base_path_csv}/test_data.csv", index=False)

# creacion de un dataset personalizado

In [ ]:

from torch.utils.data import  Dataset
image_dir = f"{base_path_img}/ISIC_2019_Training_Input/ISIC_2019_Training_Input"
class CustomDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.transform = transform
       

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.dataframe.iloc[idx]['img'] + ".jpg")
        image = Image.open(img_name)
        label = self.dataframe.iloc[idx]['cat']
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:

import torch
from torchvision import transforms
from torch.utils.data import DataLoader
from PIL import Image
import os
# Carga el conjunto de datos de entrenamiento sin ninguna normalización para calcular la media y la desviación estándar
unnormalized_transform = transforms.Compose([
    transforms.Resize((150, 150)), 
    transforms.ToTensor()
])
unnormalized_dataset = CustomDataset(data, image_dir, transform=unnormalized_transform)
loader = DataLoader(unnormalized_dataset, batch_size=64, shuffle=False)

# Calcular la media y la desviación estándar
mean_sum = torch.zeros(3)
std_sum = torch.zeros(3)
n_samples = 0

for images, _ in loader:
    # Asegúrate de que las imágenes están en el rango 0-1
    images = images / 255.0 if images.max() > 1 else images

    batch_samples = images.size(0)
    images = images.view(batch_samples, images.size(1), -1)
    mean = images.mean(dim=[0, 2])
    std = images.std(dim=[0, 2])

    mean_sum += mean * batch_samples
    std_sum += std * batch_samples
    n_samples += batch_samples

mean = mean_sum / n_samples
std = std_sum / n_samples

# Transformaciones para el conjunto de entrenamiento
train_transform = transforms.Compose([
    transforms.Resize((150, 150)),             
    transforms.RandomRotation(180),            
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

# Transformaciones para el conjunto de validación y test (sin data augmentation)
test_valid_transforms = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

train_dataset = CustomDataset(train_data,image_dir, transform=train_transform)
val_dataset = CustomDataset(val_data,image_dir, transform=test_valid_transforms)
test_dataset = CustomDataset(test_data, image_dir,transform=test_valid_transforms)

# comprobaciones
sample_dataset = CustomDataset(train_data, image_dir, transform=train_transform)

# Acceder e imprimir las primeras 5 muestras del dataset
for i in range(5):
    image, label = sample_dataset[i]
    print(f"Muestra {i}: Imagen - {type(image)}, Dimensiones - {image.size()}, Etiqueta - {label}")

# Imprimir los valores de la media y la desviación estándar
print(f"Media: {mean}, Desviación Estándar: {std}")



train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Verificar DataLoader
for images, labels in train_loader:
    print(f"Batch de imágenes: {images.shape}, Batch de etiquetas: {labels.shape}")
    break 

In [ ]:
# imports
import torch.nn as nn
import torch.nn.functional as F

## Red Neuronal simple 

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Capas convolucionales
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)

        # Max pooling
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        # Capa Dropout 
        self.dropout25 = nn.Dropout(0.25)
        self.dropout50 = nn.Dropout(0.5)

        # Capas Fully connected 
        self.fc1 = nn.Linear(in_features=128 * 18 * 18, out_features=512)
        self.fc2 = nn.Linear(in_features=512, out_features=8)  # Asegúrate de que 8 sea el número correcto de clases

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.dropout25(x)
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.dropout25(x)

        x = x.view(-1, 128 * 18 * 18)
        x = F.relu(self.fc1(x))
        x = self.dropout50(x)
        x = self.fc2(x)
        return x

model = CNN()
model.to(device)
print(model)


## Preparación del Modelo
Esta sección prepara el modelo de red neuronal usando PyTorch, incluyendo la definición de la arquitectura del modelo y los cargadores de datos.

In [ ]:
from matplotlib import pyplot as plt


def plot_loss(history):
    plt.plot(history)
    plt.xlabel('Batch')
    plt.ylabel('Loss')
    plt.show()

## Funciones de Entrenamiento y Evaluación
Aquí se definen las funciones para entrenar el modelo y evaluar su rendimiento.

In [ ]:

from sklearn import metrics


def eval(model, test_loader, device, criterion):
    model.eval()

    total = 0
    correct = 0

    pred = []
    real = []
    val_loss = 0

    with torch.no_grad():
        for imgs, cats in test_loader:
    
            imgs, cats = imgs.to(device), cats.to(device)

            outputs = model(imgs)
            _, predicted = torch.max(outputs, 1)
            
            
            val_loss += criterion(outputs, cats).item()

            pred.extend(predicted.tolist())
            real.extend(cats.tolist())

            total += cats.size(0)
            correct += (predicted == cats).sum().item()

    print(f'Accuracy on images: {100 * correct / total:.2f}%')
    print(metrics.classification_report(pred, real, zero_division=0))
    
    return torch.FloatTensor(pred), torch.FloatTensor(real), val_loss/(len(test_loader))

In [ ]:
def train_loop(model, train_loader, optimizer, criterion, num_epoch, device, history = []):

    for epoch in range(num_epoch):

        running_loss = 0
        for i, (imgs, cats) in enumerate(train_loader, 1):
            
            model.train()

            imgs, cats = imgs.to(device), cats.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, cats)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            if not i % 10:
                print(f"Epoch {epoch+1}, Batch {i}, Loss: {running_loss/10:.4f}")
                history.append(running_loss/10)
                running_loss = 0.0
                
        _, _, val_loss = eval(model, train_loader, device, criterion=criterion)

        print(f"Validation Loss: {val_loss:.4f}")
        early_stopping(val_loss)

        if early_stopping.early_stop:
            print("Early stopping")
            break

    print("Finished training")

## Utilidades Adicionales
Funciones de utilidad adicionales, como la detención temprana, se definen en esta sección.

In [ ]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

early_stopping = EarlyStopping(patience=10, min_delta=0.01)


In [ ]:
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()
num_epochs = 200

train_loop(model, train_loader, optimizer, criterion, num_epochs, device)

## Conclusión
